In [ ]:
!pip install simpletransformers
!pip install spacy==3.2.4

     |████████████████████████████████| 249 kB 32.7 MB/s 
     |████████████████████████████████| 10.1 MB 59.7 MB/s 
     |████████████████████████████████| 1.2 MB 1.1 MB/s 
     |████████████████████████████████| 346 kB 68.1 MB/s 
     |████████████████████████████████| 1.8 MB 62.3 MB/s 
     |████████████████████████████████| 4.2 MB 57.1 MB/s 
     |████████████████████████████████| 6.6 MB 56.3 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 741 kB/s 
     |████████████████████████████████| 181 kB 75.5 MB/s 
     |████████████████████████████████| 145 kB 61.2 MB/s 
     |████████████████████████████████| 63 kB 302 kB/s 
     |████████████████████████████████| 140 kB 226 kB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 1.1 MB 73.9 MB/s 
     |████████████████████████████████| 127 kB 78.9 MB/s 
     |████████████████

In [ ]:
!wget  https://www.dropbox.com/s/4w6p9raq2iw92c4/punct-dts.zip?dl=1 -O punct-dts.zip
!unzip punct-dts.zip

--2022-05-23 13:03:19--  https://www.dropbox.com/s/sukruo2u3tiq5fr/punct-data.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/sukruo2u3tiq5fr/punct-data.zip [following]
--2022-05-23 13:03:20--  https://www.dropbox.com/s/dl/sukruo2u3tiq5fr/punct-data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd0175e3d053871380218634435.dl.dropboxusercontent.com/cd/0/get/Bl0R97bFvstbm8pHwic3AabI9SFjEbM5Nnn6Uf7IiHQpBuGx3CHKhlP1l3wK90_Ox8FVrx4RyYFDW-nX2TzpxDPo3Ls-brM_esC-l4U_fxQ0KFYARRerl578JzA7BNbew3uuFjobbTwDDzZCj32txm-Xugr6RahR6eUDRwL7kXqmZg60EeV-4xiKc7VJ8i723U4/file?dl=1# [following]
--2022-05-23 13:03:20--  https://ucd0175e3d053871380218634435.dl.dropboxusercontent.com/cd/0/get/Bl0R97bFvstbm8pHwic3AabI9SFjEbM5Nnn6

In [26]:
import os
import pandas as pd

BASE_DIR = 'punct-dts/punct-data/punct-data/'

dataset = {
    
}

for filename in os.listdir(BASE_DIR):
  dataset[filename.replace('.csv', '')] = pd.read_csv(os.path.join(BASE_DIR, filename)).dropna()
 

In [ ]:
dataset['train']

,sentence_id,labels,words
0,0,O,Filmado
1,0,O,na
2,0,O,cidade
3,0,O,de
4,0,O,Nova
...,...,...,...
239072,58213,O,democracia
239073,58213,O,direta
239074,58213,O,ter
239075,58213,I-PERIOD,sucesso


In [ ]:
!pip install wandb

import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import wandb


In [ ]:

# Create a new run
project = "bert-base-cased-tedtalk"
with wandb.init(project=project) as run:

  # Connect an Artifact to the run
  model_name = "neuralmind/bert-base-portuguese-cased"
  #my_model_artifact = run.use_artifact(my_model_name)

  # Download model weights to a folder and return the path
  #model_dir = my_model_artifact.download()
  train_args = {
      'evaluate_during_training': True,
      'logging_steps': 10,
      'num_train_epochs': 15,
      'evaluate_during_training_steps': dataset['train'].shape[0],
      'save_eval_checkpoints': False,
      'train_batch_size': 16,
      'eval_batch_size': 8,
      'overwrite_output_dir': True,
      'save_eval_checkpoints':False,
      'save_model_every_epoch':False,
      'save_steps':-1,
      'labels_list':dataset['train'].labels.unique().tolist(),
      'use_early_stopping':True,
      'wandb_project':"bert-base-cased-"
  }


  model = NERModel(
      "bert",
      model_name,
      args=train_args,
      use_cuda=torch.cuda.is_available()
  )
  model.train_model(dataset['train'], eval_data=dataset['dev'])
  result, model_outputs, wrong_preds = model.eval_model(dataset['test'])

  pd.DataFrame.from_dict(result, orient='index').T.to_csv('test_result.csv', index=False,index_label=False)


In [27]:
from collections import Counter
import pandas as pd
import re 
from collections import defaultdict
from spacy.matcher import Matcher
from spacy.tokens import Span
from tqdm import tqdm
import spacy

In [29]:
nlp = spacy.blank('pt')

In [28]:
def preprocess_data(dataframe):
     

    TOTAL = len(list(dataframe.groupby("sentence_id")))
    data = []
    for _, group in tqdm(dataframe.groupby("sentence_id"), total=TOTAL):
        text = " ".join(group.words)
        doc = nlp.make_doc(text)
        ents = []
        count_dict = defaultdict(lambda : 0)
        for i, label in enumerate(group.labels):
            if label != "O":
                matching_word = group.words.tolist()[i]
                span = Span(doc, i, i+1, label=label)
                ents.append((span.start_char, span.end_char, label.replace('I-','')))
                count_dict[matching_word] += 1
        ent = (text, {
            'entities': ents
        })
        data.append(ent)
    return data

In [30]:

TEST_DATA = preprocess_data(dataset['test'])

100%|██████████| 1282/1282 [00:01<00:00, 860.10it/s]


In [31]:
from spacy.tokens import Doc
from spacy.training import Example
from tqdm.notebook import trange
#import spacy_transformers
import numpy as np
import torch

In [32]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.labels_list = ["O", "COMMA", "PERIOD", "QUESTION"]

model = NERModel(
    "bert",
    "./outputs/best_model/",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

In [98]:
y_true = []
texts = []
for _, group in dataset['test'].groupby("sentence_id"):
        text = " ".join(group.words)
        texts.append(text)
        y_true.append(group.labels.apply(lambda label: label.replace("I-","") ).tolist())

In [35]:
predictions = model.predict(texts)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/161 [00:00<?, ?it/s]

In [97]:

y_pred = []
for i,pred in enumerate(predictions[0], 1):
  y_pred.append(list(map(lambda item:list(item.values())[0].replace("I-",""), pred)))
    

In [55]:
y_pred[:1],y_true[:1]

([['O', 'O', 'O', 'O', 'O', 'COMMA']], [['O', 'O', 'O', 'O', 'O', 'QUESTION']])

In [101]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_pred_tf = mlb.fit_transform(y_pred)
mlb = MultiLabelBinarizer()
y_test_tf = mlb.fit_transform(y_true)

In [76]:
mlb.inverse_transform(y_test_tf[:1]),mlb.inverse_transform(y_pred_tf[:1])

([('O', 'QUESTION')], [('COMMA', 'O')])

In [105]:
y_test_tf[:5],y_pred_tf[:5]

(array([[0, 1, 0, 1],
        [1, 1, 1, 0],
        [1, 1, 1, 0],
        [0, 0, 1, 0],
        [1, 1, 1, 0]]),
 array([[1, 1, 0, 0],
        [1, 1, 1, 0],
        [1, 1, 1, 0],
        [1, 0, 0, 0],
        [1, 1, 1, 0]]))

In [106]:
from sklearn.metrics import classification_report

target_names = dataset['train'].labels.unique().tolist()
outdict = classification_report(y_test_tf, y_pred_tf, target_names=target_names, output_dict=True)

df = pd.DataFrame(outdict)
df.loc[:, ['I-COMMA', 'I-QUESTION', 'I-PERIOD']]

,I-COMMA,I-QUESTION,I-PERIOD
precision,0.988115,0.933333,1.0
recall,0.947111,0.507246,1.0
f1-score,0.967179,0.657277,1.0
support,1229.000000,138.000000,1271.0


In [83]:
df.to_csv('df.csv', index=False,index_label=False)

In [107]:
df.loc[:, ['I-COMMA', 'I-QUESTION', 'I-PERIOD']].to_latex()

'\\begin{tabular}{lrrr}\n\\toprule\n{} &      I-COMMA &  I-QUESTION &  I-PERIOD \\\\\n\\midrule\nprecision &     0.988115 &    0.933333 &       1.0 \\\\\nrecall    &     0.947111 &    0.507246 &       1.0 \\\\\nf1-score  &     0.967179 &    0.657277 &       1.0 \\\\\nsupport   &  1229.000000 &  138.000000 &    1271.0 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [ ]:
text = "Eu gostaria de ir para casa agora mas eu não sei como"

In [ ]:
list(predictions[0][0][-1].values())[0]

'COMMA'

In [ ]:
from spacy.tokens import Span

predictions_ner = predictions[0]
doc = nlp(texts[0])
doc.set_ents([Span(doc, i, i+1, "I-"+list(item.values())[0]) 
               for i, item in enumerate(predictions_ner[0]) 
                if list(item.values())[0] !="O"])
ents = list(doc.ents)
ents[0].label_

'I-COMMA'

In [ ]:
doc[:2]

Mr. Best

In [67]:
examples = []
for i, (text, entities) in enumerate(TEST_DATA):

    doc = nlp(text)
    doc.set_ents([Span(doc, i, i+1, list(item.values())[0]) 
               for i, item in enumerate(predictions_ner[i]) 
                if list(item.values())[0] !="O"])
  
    example = Example.from_dict(doc, entities)
    examples.append(example)

In [61]:
example

{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-PERIOD', 'O', 'O', 'O', 'O', 'U-PERIOD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-COMMA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-PERIOD'], 'links': {}}, 'token_annotation': {'ORTH': ['De', 'forma', 'alguma', 'os', 'protestos', 'serão', 'em', 'vão', 'Já', 'é', 'possível', 'ver', 'resultados', 'Neste', 'momento', 'é', 'muito', 'importante', 'que', 'o', 'povo', 'saiba', 'o', 'que', 'quer', 'e', 'lute', 'até', 'conseguir', 'mas', 'milhões', 'de', 'pessoas', 'nas', 'ruas', 'nunca', 'irão', 'substituir', 'o', 'voto', 'consciente'], 'SPACY': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False], 'TAG': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [58]:
entities

{'entities': [(38, 41, 'PERIOD'),
  (60, 70, 'PERIOD'),
  (143, 152, 'COMMA'),
  (214, 224, 'PERIOD')]}

In [ ]:
TEST_DATA[:1]

[('Você não adora uma boa soneca', {'entities': [(23, 29, 'QUESTION')]})]

In [ ]:
TEST_DATA[2]

('E continuamos a investir Investimos no Devin cuja mãe teve que escolher entre pagar o aluguel ou as contas médicas Ele descobriu seu amor pela pintura na ChopArt Investimos na Liz que tem vivido nas ruas pela maior parte da adolescência e que usa a música para se centrar quando seus traumas parecem grandes demais para a sua idade Investimos na Maria que usa a poesia na recuperação após seu avô ter morrido na van onde mora o resto de sua família',
 {'entities': [(16, 24, 'PERIOD'),
   (39, 44, 'COMMA'),
   (107, 114, 'PERIOD'),
   (154, 161, 'PERIOD'),
   (176, 179, 'COMMA'),
   (224, 236, 'COMMA'),
   (326, 331, 'PERIOD'),
   (346, 351, 'COMMA'),
   (441, 448, 'PERIOD')]})

In [ ]:
predictions_ner[2]

In [ ]:
predictions_ner[:1]

[[{'Você': 'O'},
  {'não': 'O'},
  {'adora': 'O'},
  {'uma': 'O'},
  {'boa': 'O'},
  {'soneca': 'COMMA'}]]

In [66]:
from spacy.scorer import Scorer
scorer = Scorer(nlp)
scores = Scorer.score_spans(examples, "ents")
print(scores["ents_f"])

0.0


In [ ]:
!zip bert_ner.zip -r outputs/best_model

  adding: outputs/best_model/ (stored 0%)
  adding: outputs/best_model/vocab.txt (deflated 52%)
  adding: outputs/best_model/config.json (deflated 55%)
  adding: outputs/best_model/model_args.json (deflated 60%)
  adding: outputs/best_model/eval_results.txt (deflated 21%)
  adding: outputs/best_model/tokenizer_config.json (deflated 36%)
  adding: outputs/best_model/pytorch_model.bin (deflated 7%)
  adding: outputs/best_model/training_args.bin (deflated 48%)
  adding: outputs/best_model/scheduler.pt (deflated 49%)
  adding: outputs/best_model/optimizer.pt (deflated 19%)
  adding: outputs/best_model/special_tokens_map.json (deflated 40%)
